# Scrape KIT Mensa Website
This tool allows you to scrape data about your mensa activity at the Karlsruher Institute of Technology, sets up a dataframe and saves everything as a .scv <br>

What you'll need:
- AUTOLOAD has to be activated at the studierendenservice and the login data obtained!
- save the login data in the login.txt in same directory or rewrite the corresponding comment
- install and save chromedriver in the same directory

What you should know:
I have no clue how to properly code, as mechatronics student I just wanted to learn how to scrape a website. This is the outcome. 

### Import requests:

In [37]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import datetime

### Login:
Open login page:

In [38]:
cwd = os.getcwd()
DRIVER_PATH = cwd+'\chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
#ignore warning or fix the code: https://stackoverflow.com/questions/64717302/deprecationwarning-executable-path-has-been-deprecated-selenium-python
login_url = "https://autoload.sw-ka.de/KartenService/index.html#home"
url = "https://autoload.sw-ka.de/KartenService/index.html#trans"
driver.get(login_url)

C:\Users\loren\AppData\Local\Temp/ipykernel_14316/2703279490.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


Login data from textfile:

In [39]:
with open('login.txt', 'r') as f:
    text = f.read()
    f.close()
head, sep, tail = text.partition("login_pw = '")
login_num = int(head.replace("login_num = "," "))
login_pw = str(tail)
login_pw = login_pw[:-1]
#uncomment for use without login.txt   
#login_num = 123456
#login_pw = 'abcdefghij'


Actual login:

In [40]:
WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, ".//*[@type='submit']")))
password_input = driver.find_element(By.XPATH, "//input[@type='password']")
password_input.send_keys(login_pw)
username_input = password_input.find_element(By.XPATH, ".//preceding::input[not(@type='hidden')]")
username_input.send_keys(login_num)
form_element = password_input.find_element(By.XPATH, ".//ancestor::form")
submit_button = form_element.find_element(By.XPATH, ".//*[@type='submit']").click()
WebDriverWait(driver, timeout=10).until(lambda driver: driver.current_url == "https://autoload.sw-ka.de/KartenService/index.html#info")

True

Switch to transactions page:

In [41]:

driver.get(url)
driver.implicitly_wait(1)
#start date (use min):
transactionDateMin = driver.find_element(By.XPATH,'//*[@id="id_32_gen"]').get_attribute("min")
transactionDateMin = datetime.datetime.strptime(transactionDateMin, '%Y-%m-%d').strftime('%d/%m/%Y')
#transactionDateMin = '25/04/2022'   #overwrite to specify a desired date
start_date = driver.find_element(By.XPATH,'//*[@id="id_32_gen"]').send_keys(transactionDateMin)
#end date (use max):
transactionDateMax = driver.find_element(By.XPATH,'//*[@id="id_32_gen"]').get_attribute("max")
transactionDateMax = datetime.datetime.strptime(transactionDateMax, '%Y-%m-%d').strftime('%d/%m/%Y')
end_date = driver.find_element(By.XPATH,'//*[@id="id_34_gen"]').send_keys(transactionDateMax)
submit_button = driver.find_element(By.XPATH, '//*[@id="transactionFormSubmit"]').click()

### Harvest data:

not the most elegant solution probably but I was confused on how to import nested tables from html.. <br>
Shows how many transactions are available:

In [42]:
#setup dataframe
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="transactionTable"]/tbody[1]/tr[1]/td[1]/button')))
tbl = driver.find_element(By.XPATH, '//*[@id="transactionTable"]').get_attribute('innerHTML')
table_list = pd.read_html(tbl)
n_transactions = len(table_list)
print('Transactions found:', n_transactions)
header_text = driver.find_element(By.XPATH,'/html/body/div[4]/form/div[1]/div[2]/table/thead').get_attribute('outerText')
header_0 = list(' '.join(BeautifulSoup(header_text, "html.parser").stripped_strings).split("\t"))
header_1 = list(table_list[0].columns.values)
header_1.remove('Unnamed: 0')
header = [header_0[0]]+header_1+header_0[1:len(header_0)]
df = pd.DataFrame(columns=header,index=range(len(table_list)))

Transactions found: 167


Format dataframe:<br>
The html page has nested tables for the transactions. <br> In the df each transaction has an index, while each item in one transaction (cart) gets the same index, timestamp, ...

In [43]:
transaction=[]
transaction_text=[]
for i in range(1,len(table_list)+1):
    transaction.append(driver.find_element(By.XPATH, '/html/body/div[4]/form/div[1]/div[2]/table/tbody['+str(i)+']/tr[1]').get_attribute('outerText'))
    transaction_text.append(list(transaction[i-1].split("\t")))
df_transaction_text = pd.DataFrame(transaction_text)
df_transaction_text = df_transaction_text.replace(r'\n',' ', regex=True) 
df['Datum'] = df_transaction_text[1]
df['Ort'] = df_transaction_text[2]
df['Kasse'] = df_transaction_text[3]
df['Typ'] = df_transaction_text[4]
df['Bon-Nr.'] = df_transaction_text[5]
df['Bonus'] = df_transaction_text[6]
df['Betrag'] = df_transaction_text[7]
df['PDF'] = df_transaction_text[8]

#dict of df (carts) for each transaction
cart_dic = dict()
for i in range(1,n_transactions+1):    
    cart_text_list = pd.read_html(driver.find_element(By.XPATH, '/html/body/div[4]/form/div[1]/div[2]/table/tbody['+str(i)+']/tr[2]/td[2]/table').get_attribute('outerHTML'))
    cart_text = cart_text_list[0]
    cart_text = cart_text.drop('Unnamed: 0',axis=1)
    #cart_text = cart_text.replace(r'\n',' ', regex=True) #no need
    cart_dic[i] = cart_text
cart_df = pd.concat(cart_dic, axis=0)

#logout of website
logout_button = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/a[2]').click()
driver.close()

#repeat transaction to match cart length, insert cart df in df
repeat_list = []
for i in range(1,n_transactions+1):
        repeat_list.append(len(cart_dic[i]))
df = df.loc[df.index.repeat(repeat_list)]
cart_df = pd.concat(cart_dic, axis=0)
cart_df.reset_index(inplace=True)
df.reset_index(inplace=True)
df[['Artikel', 'Menge', 'Preis', 'Rabatt', 'Gesamt']] = cart_df[['Artikel', 'Menge', 'Preis', 'Rabatt', 'Gesamt']]

### Save data to .csv

In [44]:
df.to_csv('mensascrape.csv', encoding='utf-8')

### take a look

In [45]:
df.head(5)

,index,Datum,Artikel,Menge,Preis,Rabatt,Gesamt,Ort,Kasse,Typ,Bon-Nr.,Bonus,Betrag,PDF
0,0,30.05.2022 14:06,Club Mate,1,1.20 €,NaN,1.20 €,Küche Mensa Am Adenauer,61012 Cafeteria Theke,Verkauf,61012-3903-808,,-1.35 €,PDF anzeigen
1,0,30.05.2022 14:06,"Pfand + 0,15 €",1,0.15 €,NaN,0.15 €,Küche Mensa Am Adenauer,61012 Cafeteria Theke,Verkauf,61012-3903-808,,-1.35 €,PDF anzeigen
2,1,30.05.2022 13:07,Kalbscurrywurst mild Mensa,1,2.00 €,NaN,2.00 €,Koeriwerk A7,61014 [koeri]werk,Verkauf,61014-3508-602,,-4.94 €,PDF anzeigen
3,1,30.05.2022 13:07,Pommes Frites Mensa,1,1.30 €,NaN,1.30 €,Koeriwerk A7,61014 [koeri]werk,Verkauf,61014-3508-602,,-4.94 €,PDF anzeigen
4,1,30.05.2022 13:07,Schale 130g,1,1.64 €,NaN,1.64 €,Koeriwerk A7,61014 [koeri]werk,Verkauf,61014-3508-602,,-4.94 €,PDF anzeigen
